In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
tf.reset_default_graph()

In [ ]:
from tensorflow.python.compiler import tensorrt as trt

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import time

import scipy.io
import numpy as np
import tensorflow as tf
import h5py
import random
import matplotlib.pyplot as plt
import os
import cv2
import itertools
from tensorflow.python.layers.core import Dense
#from tensorflow.contrib import layers
from tensorflow.python.layers.core import Dense

%matplotlib inline

In [ ]:
def load_Images(Data_Train_Path):
    main_path=[f for f in np.sort(os.listdir(Data_Train_Path)) if f.endswith('redraww')]
    #height =547
    imgs=[]
    for s in main_path:
        sub=Data_Train_Path+s
        sub_path= [f for f in np.sort(os.listdir(sub))if f.endswith('.tif')]
        for t in sub_path:
            im_path=sub + '/' +t
            img=cv2.imread(im_path,cv2.IMREAD_UNCHANGED)
            img=cv2.resize(img,(64,64))
            imgs.append(img)
            #plt.imshow(img)
            #plt.show()
            #print('expend image ',np.expand_dims(imgs, -1))
            
    return np.expand_dims(imgs, -1)

In [ ]:
#Input 
def load_Coordinates(Data_Train_Path):
    Data_Label = []
    DATA_Label_K=[]
    main_path = [f for f in np.sort(os.listdir(Data_Train_Path)) if f.endswith('onlinee')]
    
    for s in main_path:
        sub = Data_Train_Path + s        
        sub_path = [f for f in np.sort(os.listdir(sub)) if f.endswith('.txt')]
        for t in sub_path:
            txt_path = sub + '/' +t
            file_B = open(txt_path, 'r')
            data = file_B.read()
            x = data.split('\n')#
            
            L=len(x)-1
           
            online_data = np.ones([50,2],np.int32)
            online_data_K = np.ones([51,2],np.int32)
          
            y=0
          
            while y <= 50  :               
                z = x[y].split(',') 
                if z == ['']:
                    break
                
               
                online_data[y] = [int(z[0]),int(z[1])]
                online_data_K[y+1] = [int(z[0]),int(z[1])]
                y+=1
            online_data_K = online_data_K[:50,:]
            file_B.close()
            Data_Label.append(online_data)
            DATA_Label_K.append(online_data_K) 
          
           
    return np.array(Data_Label), np.array(DATA_Label_K)

In [ ]:
def Architecture(Input, outputs, is_training = True):
             
    with tf.variable_scope("Network"):
        
        with tf.variable_scope("CNN"):
            conv1 = tf.layers.conv2d(Input,64,3,activation=tf.nn.relu,name = 'conv1', padding = 'same')
            pool1 = tf.layers.max_pooling2d(conv1,2,[2,2],name = 'pool1')

            conv2 = tf.layers.conv2d(pool1,128,3,activation=tf.nn.relu,name = 'conv2', padding = 'same')
            pool2 = tf.layers.max_pooling2d(conv2, 2,[2,1],name = 'pool2',  padding = 'same')


            conv3 = tf.layers.conv2d(pool2,256,3,activation=None,name = 'conv3', padding = 'same')
            b_norm3 = tf.nn.relu(tf.layers.batch_normalization(conv3, training = is_training, name='batch-norm1'), name = 'relu3')


            conv4 = tf.layers.conv2d(b_norm3,256,3,activation=tf.nn.relu,name = 'conv4', padding = 'same')
            pool4 = tf.layers.max_pooling2d(conv4, 2,[2,1],name = 'pool4',  padding = 'same')

            conv5 = tf.layers.conv2d(pool4,256,3,activation=tf.nn.relu,name = 'conv5', padding = 'same')
            pool5 = tf.layers.max_pooling2d(conv5, 2,[2,1],name = 'pool5',  padding = 'same')

            conv6 = tf.layers.conv2d(pool5,256,3,activation=None,name = 'conv6', padding = 'same')
            b_norm6 = tf.nn.relu(tf.layers.batch_normalization(conv6, training = is_training, name='batch-norm2'), name = 'relu6')

            conv7 = tf.layers.conv2d(b_norm6,256,3,activation=tf.nn.relu,name = 'conv7', padding = 'same')
            pool7 = tf.layers.max_pooling2d(conv7, 2,[2,1],name = 'pool7',  padding = 'same')

            conv8 = tf.layers.conv2d(pool7,256,2,activation=None, name = 'conv8')
            print('conv8',conv8.shape)
            b_norm8 = tf.nn.relu(tf.layers.batch_normalization(conv8, training = is_training, name='batch-norm3'), name = 'relu8')
            print('b_norm8',b_norm8.shape)

            shape = b_norm8.get_shape().as_list()
            transposed = tf.transpose(b_norm8, perm=[0, 2, 1, 3], name='transposed') 
            conv_reshaped = tf.reshape(transposed, [ shape[0], -1, shape[1] * shape[3] ], name='reshaped') 
            print('conv_reshaped',conv_reshaped.shape)
        num_units = 256
        num_layers = 3
        
        cell_stack_encoder = []
        cell_stack_decoder = []

        with tf.variable_scope("Encoder"):
            for i in range(num_layers):
                cell_stack_encoder.append(tf.compat.v1.nn.rnn_cell.BasicRNNCell(num_units))
            encoder_cell=tf.compat.v1.nn.rnn_cell.MultiRNNCell(cell_stack_encoder, state_is_tuple=True)
            _, encoder_state=tf.compat.v1.nn.dynamic_rnn(encoder_cell, conv_reshaped, dtype=tf.float32) 

        with tf.variable_scope("Decoder"):
            for i in range(num_layers):
                cell_stack_decoder.append(tf.compat.v1.nn.rnn_cell.BasicRNNCell(num_units))
            decoder_cell=tf.compat.v1.nn.rnn_cell.MultiRNNCell(cell_stack_decoder)
            
            dec_outputs,_=tf.compat.v1.nn.dynamic_rnn(decoder_cell,outputs, initial_state=encoder_state, dtype=tf.float32) 
            
            logits = tf.layers.dense(dec_outputs, 2, name  = 'logits')  
        
            return logits    

In [ ]:
def Loss_function(Dec_Outputs, Targets):
    loss = tf.reduce_mean(tf.abs(Dec_Outputs - Targets), name = 'loss')
    return loss

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
eps=1e-8
from datetime import datetime
from tensorflow.python.framework import ops
ops.reset_default_graph()
global sess
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session (config=config)
graph = tf.get_default_graph()
import os

BatchSize = 32                                           #
learning_rate = 0.0001 
num_epochs = 100

model_path = "gdrive/My Drive/data/lstm-not-attention6_layer.ckpt-1"


LabelTrain_Targets, LabelTrain_Outputs = load_Coordinates ('gdrive/My Drive/data/')                              #
DataTrain = load_Images('gdrive/My Drive/data/')                                                                 #
print('LabelTrain_Targets shape',LabelTrain_Targets.shape)
print('LabelTrain_Outputs shape',LabelTrain_Outputs.shape)
  
print('DataTrain shape',DataTrain.shape)
Data_Train=DataTrain                                                                                      #
LabelTrain_Outputs=LabelTrain_Outputs    
print('LabelTrain_Outputs dtype',LabelTrain_Outputs.dtype)
LabelTrain_Targets=LabelTrain_Targets                                                                     #
##################################################Fin Import Data#########################################
     
###############################################Create tensors of variables################################    
Inputs = tf.placeholder(tf.float32, shape = [BatchSize, 64, 64, 1],name = 'Inputs')
outputs = tf.placeholder(tf.float32, shape = [BatchSize, None, 2],name = 'outputs')
seq_length = tf.placeholder(tf.int32, [None])
targets = tf.placeholder(tf.float32, shape = [BatchSize, None, 2],name = 'targets')

############################################### Fin Create tensors of variables################################  

logits = Architecture(Inputs, outputs)
print('targets shape',targets.shape)


#############################################train accuracy#####################################################
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(targets, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

################################################################################################################
loss = Loss_function2(logits, targets)


  
training_step = tf.train.AdamOptimizer(learning_rate = 0.0001).minimize(loss)

    
init = tf.global_variables_initializer()
saver = tf.train.Saver()
sess.run(init)

saver.restore(sess, model_path)


train_loss_list = []

mean_loss_list = []
    
timeperepoch = []
    


[Total_Data, row, col, depth] = np.shape(Data_Train)    
itr = 0
for i in range(num_epochs): 
      
    index = np.random.permutation(Total_Data)
       
    Data_Train = Data_Train[index,:,:,:]
    
    LabelTrain_Targets = LabelTrain_Targets[index,:,:]
    LabelTrain_Outputs = LabelTrain_Outputs[index,:,:]
        
    for k in range(Total_Data//BatchSize):
        start = time.time()
            
        batch_input = Data_Train[k*BatchSize:(k+1)*BatchSize,:,:,:]
        batch_outputs = LabelTrain_Outputs[k*BatchSize:(k+1)*BatchSize,:,:]
      
        batch_targets = LabelTrain_Targets[k*BatchSize:(k+1)*BatchSize,:,:]
        train_feed_dict = {Inputs:batch_input, outputs:batch_outputs, targets:batch_targets, seq_length: seq_length_batch}
       
            
        _,train_loss,logits_Out = sess.run([training_step, loss,logits], train_feed_dict)
        
        

               
        
        acc= sess.run([accuracy], feed_dict={logits: logits_Out,targets:batch_targets})
                                                       
                    
        
        itr = itr + 1
            
        train_loss_list.append(train_loss)
        end = time.time()
        timeperepoch.append(end-start)
      
        print('### L1 loss TRAINING ###')
        print ('Epoch{:3} : Step:{:3} : Train loss is :->{:>6.3f} Epoch duration :{:>6.3f}s'.format(i,k,train_loss,end- start)+ '\n')
        print(" accuracy_train"+str(acc))
        print('### fin L1 loss TRAINING ###')
    saver = tf.train.Saver()    
    
      
        
        ################################################Fin Train#####################################
  
save_path = saver.save(sess, model_path)
print("Model saved in file: %s" % save_path)
end_after_epoch=time.time() 

